# 第9章　実践編4：エピジェネティクスを含む多階層の統合によるがん研究

- 浅田　健
- 浜本隆二

##### 入力9-1

In [ ]:
# ライブラリのインポート
import argparse
from cmath import e
import copy
import glob
import math
import pwd
from tkinter import E
import matplotlib.pyplot as plt
import numpy as np
import os
 
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from pathlib import Path
from torch.utils.data import DataLoader

##### 入力9-2

In [ ]:
class AEDataset():
    def __init__(self, X, y):
        self.X = torch.Tensor(X.values)
        self.y = torch.Tensor(y.values)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

##### 入力9-3

In [ ]:
# エンコーダの定義
class Encoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_size, encoding_dim[0])
        self.pool1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(encoding_dim[0], encoding_dim[1])
        self.pool2 = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.pool1(x)
        x = torch.tanh(self.fc2(x))
        x = self.pool2(x)
        return x

# デコーダの定義
class Decoder(nn.Module):
    def __init__(self, encoding_dim, input_size):
        super().__init__()
        self.fc3 = nn.Linear(encoding_dim[1], encoding_dim[0])
        self.fc4 = nn.Linear(encoding_dim[0], input_size)

    def forward(self, x):
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        return x

class AutoEncoder(nn.Module):
    def __init__(self, input_size=13767, encoding_dim=[500, 100]):
        super().__init__()
        self.encoder = Encoder(input_size, encoding_dim)
        self.decoder = Decoder(encoding_dim, input_size)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

##### 入力9-4

In [ ]:
# トレーニング関数
def train_model(model, loss, optimizer, data_loader, epochs, fout, device):

    # 初期化
    pkl_queue = deque()
    best_loss = 100.0
    best_epoch = 0
    best_model_weights = model.state_dict()
    since = time.time()
    end = time.time()

    print(model, "\n")

    # エポックに対するループ処理
    for epoch in range(epochs):
        print("Epoch:{}/{}".format(epoch+1, epochs), end="")
        print("Epoch:{}/{}".format(epoch+1, epochs), end="", file=fout)

        # データセットに対するループ処理(ここではtrainデータセットに対してのみ)
        for phase in ["train"]:
            model.train(True)

            # データの指定
            data = data_loader[phase]

            # 初期化
            running_loss = 0

            # ミニバッチに対するループ処理
        for idx, (data_train, target_train) in enumerate(data):
            optimizer.zero_grad()
            x, y = data_train.to(device), target_train.to(device)

            with torch.set_grad_enabled(phase == "train"):
                y_pred = model(x)
                l = loss(y_pred, y)
                l.backward()
                optimizer.step()
            # 損失のアップデート
            running_loss += l.item()

        # MSEをトラッキング
        epoch_loss = running_loss / (len(data)/len(x))
    
        # 最も損失が低かったモデルを保存
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_epoch = epoch
            best_model_weights = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), "{}_epoch{}.pkl".format(fout.name.split(". txt")[0], epoch+1))
            pkl_queue.append("{}_epoch{}.pkl".format(fout.name.split(".txt")[0], epoch+1))
            if len(pkl_queue) > 1:
                pkl_file = pkl_queue.popleft()
                os.remove(pkl_file)
        
        # 予測の打ち出し
        print("\t{} Loss: {:.4f} Time: {:.4f}".format(phase, epoch_loss, time. time()-end), end="")
        print("\t{} Loss: {:.4f} Time: {:.4f}".format(phase, epoch_loss, time. time()-end), end="", file=fout)
        print("\n", end="")
        print("\n", end="", file=fout)

        end = time.time()

    # トレーニング結果を表示
    time_elapsed = time.time() - since
    print("\nTraining completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best loss: {:.4f} at epoch {}".format(best_loss, best_epoch))

##### 入力9-5

In [ ]:
def plot_loss(file, name):

    # トレーニングログファイルの読み込み
    df = pd.read_csv(file, header=None, sep=r"\s+")

    # 線グラフを作成
    fig, ax = plt.subplots()
    plt.plot(range(len(df)), df.iloc[:, 3])
    ax.set_title(f"MSE loss for \n{file}")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("MSE loss")
    fig.savefig(f"{name}.png")
    plt.close(fig)

##### 入力9-6

In [ ]:
def eval_model(model, data_loader, device):

    # 初期化 
    running_mse = 0
    preds = []

    # データとモデルをセット
    data = data_loader["test"]
    model.eval()

    # ミニバッチに対するループ処理
    for data_test, target_test in data:
        x, y = data_test.to(device), target_test.to(device)

    # 予測
    with torch.no_grad():
        y_pred = model(x)

        # MSEの計算
        squared_error = ((y_pred - x)*(y_pred - x)).sum().data
        running_mse += squared_error

        preds.append(y_pred[0])
    
    # 予測スコアを表示
    preds = np.vstack(preds)
    mse = math.sqrt(running_mse / len(data))
    print(f"MSE: {mse}")
    
    return preds

##### 入力9-7

In [ ]:
## このセルは抜粋したものであるため，単独で実行してもエラーが出るので注意されたい

# オートエンコーダモデルのロード
model = AutoEncoder(13767, [500, 100])
model = model.to(device)

# トレーニングデータセットの作成
AEdata = AEDataset(X, X)
train_data = DataLoader(AEdata, batch_size=args.batch, shuffle=True)
test_data = DataLoader(AEdata, batch_size=args.batch, shuffle=False)
data_loader = {"train": train_data, "test": test_data}

# モデルをトレーニングするための設定
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.decay, momentum=args. momentum, nesterov=True)

# トレーニングログファイルの設定
train_log = f"AE_lr{args.lr}_decay{args.decay}_momentum{args.momentum}_epochs{args.epochs}_batch{args.batch}.txt"
fout = open(check_path(str(Path(path, args.output, "AutoEncoder", train_log))), "w")

# オートエンコーダのトレーニング
train_model(model, loss, optimizer, data_loader, args.epochs, fout, device)

fout.close()

# トレーニング損失の書き出し
plot_loss(str(Path(path, args.output, "AutoEncoder", train_log)), check_path(str(Path(path, args.output, "AutoEncoder", f"LUAD_epoch{args.epochs}_loss"))))

# 最も性能の良いオートエンコーダモデルの読み込み
trained_model = glob.glob(str(Path(path, args.output, "AutoEncoder", "{}_*.pkl".format(train_log.split(".txt")[0]))))[0]
model.load_state_dict(torch.load(trained_model), strict=False)

# 学習済みモデルを用いた予測
decoded_result = eval_model(model, data_loader, device)
print(decoded_result)

# 最も良い学習済みモデルからの特徴抽出
encoded_bottleneck = model.encoder(torch.Tensor(X.values)).detach().numpy()
print(encoded_bottleneck)

# 予測結果の保存
np.savetxt(check_path(str(Path(path, args.output, "Encoder", f"LUAD_epoch{args.epochs}_std_mmRNA.csv"))), encoded_bottleneck, delimiter=",")
np.save(check_path(str(Path(path, args.output, "Encoder", f"LUAD_epoch{args.epochs}_std_mmRNA.npy"))), encoded_bottleneck)

##### 入力9-8

In [ ]:
%%writefile Autoencoder.py

# ライブラリのインポート
import argparse
from cmath import e
import copy
import glob
import math
import pwd
from tkinter import E
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim

from collections import deque
from pathlib import Path
from torch.utils.data import DataLoader

def check_path(filename):

    if not Path(filename).is_dir():
        Path(filename).parents[0].mkdir(parents=True, exist_ok=True)

    return filename


class AEDataset():
    def __init__(self, X, y):
        self.X = torch.Tensor(X.values)
        self.y = torch.Tensor(y.values)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# エンコーダの定義
class Encoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_size, encoding_dim[0])
        self.pool1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(encoding_dim[0], encoding_dim[1])
        self.pool2 = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.pool1(x)
        x = torch.tanh(self.fc2(x))
        x = self.pool2(x)
        return x

# デコーダの定義
class Decoder(nn.Module):
    def __init__(self, encoding_dim, input_size):
        super().__init__()
        self.fc3 = nn.Linear(encoding_dim[1], encoding_dim[0])
        self.fc4 = nn.Linear(encoding_dim[0], input_size)

    def forward(self, x):
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        return x

class AutoEncoder(nn.Module):
    def __init__(self, input_size=13767, encoding_dim=[500, 100]):
        super().__init__()
        self.encoder = Encoder(input_size, encoding_dim)
        self.decoder = Decoder(encoding_dim, input_size)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# トレーニング関数
def train_model(model, loss, optimizer, data_loader, epochs, fout, device):

    # 初期化
    pkl_queue = deque()
    best_loss = 100.0
    best_epoch = 0
    best_model_weights = model.state_dict()
    since = time.time()
    end = time.time()

    print(model, "\n")

    # エポックに対するループ処理
    for epoch in range(epochs):
        print("Epoch:{}/{}".format(epoch+1, epochs), end="")
        print("Epoch:{}/{}".format(epoch+1, epochs), end="", file=fout)

        # データセットに対するループ処理 (ここではtrainデータセットに対してのみ)
        for phase in ["train"]:
            model.train(True)

            # データの指定
            data = data_loader[phase]

            # 初期化
            running_loss = 0

            # ミニバッチに対するループ処理
            for idx, (data_train, target_train) in enumerate(data):
                optimizer.zero_grad()
                x, y = data_train.to(device), target_train.to(device)

                with torch.set_grad_enabled(phase == "train"):
                    y_pred = model(x)
                    l = loss(y_pred, y)

                    l.backward()
                    optimizer.step()

                # 損失のアップデート
                running_loss += l.item()

            # MSEをトラッキング
            epoch_loss = running_loss / (len(data)/len(x))

            # 最も損失が低かったモデルを保存
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                best_model_weights = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "{}_epoch{}.pkl".format(fout.name.split(".txt")[0], epoch+1))
                pkl_queue.append("{}_epoch{}.pkl".format(fout.name.split(".txt")[0], epoch+1))
                if len(pkl_queue) > 1:
                    pkl_file = pkl_queue.popleft()
                    os.remove(pkl_file)

            #予測の打ち出し
            print("\t{} Loss: {:.4f} Time: {:.4f}".format(phase, epoch_loss, time.time()-end), end="")
            print("\t{} Loss: {:.4f} Time: {:.4f}".format(phase, epoch_loss, time.time()-end), end="", file=fout)
            print("\n", end="")
            print("\n", end="", file=fout)

            end = time.time()

    # トレーニング結果を表示
    time_elapsed = time.time() - since
    print("\nTraining completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best loss: {:.4f} at epoch {}".format(best_loss, best_epoch))


def plot_loss(file, name):

    # トレーニングログファイルの読み込み
    df = pd.read_csv(file, header=None, sep=r"\s+")

    # 線グラフを作成
    fig, ax = plt.subplots()
    plt.plot(range(len(df)), df.iloc[:, 3])
    ax.set_title(f"MSE loss for \n{file}")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("MSE loss")
    fig.savefig(f"{name}.png")
    plt.close(fig)

def eval_model(model, data_loader, device):

    # 初期化
    running_mse = 0
    preds = []

    # データとモデルをセット
    data = data_loader["test"]
    model.eval()

    # ミニバッチに対するループ処理
    for data_test, target_test in data:
        x, y = data_test.to(device), target_test.to(device)

        # 予測
        with torch.no_grad():
            y_pred = model(x)

            # MSEの計算
            squared_error = ((y_pred - x)*(y_pred - x)).sum().data
            running_mse += squared_error

            preds.append(y_pred[0])

    # 予測スコアを表示
    preds = np.vstack(preds)
    mse = math.sqrt(running_mse / len(data))
    print(f"MSE: {mse}")

    return preds


def parseArgs():
    
    parser = argparse.ArgumentParser(description="Train autoencoder model")

    # 必要な引数
    parser.add_argument("-i", "--input", type=str, required=True, help="Input data to train the model on")

    # オプショナルな引数
    parser.add_argument("-b", "--batch", type=int, default=1, help="Training minibatch size (default: 1)")
    parser.add_argument("-e", "--epochs", type=int, default=150, help="Training epochs (default: 150)")

    parser.add_argument("-l", "--lr", type=float, default=0.01, help="Optimizer learning rate (default: 0.01)")
    parser.add_argument("-d", "--decay", type=float, default=1e-6, help="Optimizer decay rate (default: 1e-6)")
    parser.add_argument("-m", "--momentum", type=float, default=0.9, help="Optimizer momentum (default: 0.9)")

    parser.add_argument("-o", "--output", type=str, default="AE_LUAD_PyTorch", help="Output directory name (default: AE_LUAD_PyTorch)")

    # 引数のコマンドラインにおける打ち出し
    args = parser.parse_args()

    print("Called with args:")
    print(f"{args}\n")

    return args


def main():

    # コマンドライン引数をパースする
    args = parseArgs()

    # 変数の初期化
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    path = Path.cwd()

    # インプットデータをデータフレームとして読み込み
    df = pd.read_csv(Path(path, args.input), header=0, usecols=lambda c: c not in ["GeneSymbol", "Platform"])
    df = df.transpose()
    X = df.astype(np.float32)

     # オートエンコーダモデルのロード
    model = AutoEncoder(13767, [500, 100])
    model = model.to(device)

    # トレーニングデータセットの作成
    AEdata = AEDataset(X, X)
    train_data = DataLoader(AEdata, batch_size=args.batch, shuffle=True)
    test_data = DataLoader(AEdata, batch_size=args.batch, shuffle=False)
    data_loader = {"train": train_data, "test": test_data}
    
    # モデルをトレーニングするための設定
    loss = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.decay, momentum=args.momentum, nesterov=True)

    # トレーニングログファイルの設定。
    train_log = f"AE_lr{args.lr}_decay{args.decay}_momentum{args.momentum}_epochs{args.epochs}_batch{args.batch}.txt"
    fout = open(check_path(str(Path(path, args.output, "AutoEncoder", train_log))), "w")

    # オートエンコーダのトレーニング
    train_model(model, loss, optimizer, data_loader, args.epochs, fout, device)

    fout.close()

    # トレーニング損失の書き出し
    plot_loss(str(Path(path, args.output, "AutoEncoder", train_log)), check_path(str(Path(path, args.output, "AutoEncoder", f"LUAD_epoch{args.epochs}_loss"))))

    # 最も性能の良いオートエンコーダモデルの読み込み
    trained_model = glob.glob(str(Path(path, args.output, "AutoEncoder", "{}_*.pkl".format(train_log.split(".txt")[0]))))[0]
    model.load_state_dict(torch.load(trained_model), strict=False)

    # 学習済みモデルを用いた予測
    decoded_result = eval_model(model, data_loader, device)
    print(decoded_result)

    # 最も良い学習済モデルからの特徴抽出
    encoded_bottleneck = model.encoder(torch.Tensor(X.values)).detach().numpy()
    print(encoded_bottleneck)

    # 予測結果の保存
    np.savetxt(check_path(str(Path(path, args.output, "Encoder", f"LUAD_epoch{args.epochs}_std_mmRNA.csv"))), encoded_bottleneck, delimiter=",")
    np.save(check_path(str(Path(path, args.output, "Encoder", f"LUAD_epoch{args.epochs}_std_mmRNA.npy"))), encoded_bottleneck)


if __name__ == "__main__":
    main()

##### 入力9-9

In [ ]:
!ls

##### 入力9-10

In [ ]:
!ls

##### 入力9-11

In [ ]:
!python Autoencoder.py -i input_file_yodosya_zikkenigaku.csv

##### 入力9-12

In [ ]:
!python Autoencoder.py -i input_file_yodosya_zikkenigaku.csv -e 10 -o test/epoch10/

##### 入力9-13


In [ ]:
conda info -e

##### 入力9-14

In [ ]:
python3 Autoencoder.py -i input_file_yodosya_zikkenigaku.csv